# Задание Lite к занятию Методы Faiss, Langchain

Возьмите базу знаний "Правила безопасности опасных производственных объектов"

URL: https://docs.google.com/document/d/1q4l912Re8zuIfBax4FDS3ZppYmVPzER3Si2wrmznddc

или любой другой текстовый документ по ссылке с ГуглДиска.

- Разбейте текст на чанки с использованием CharacterTextSplitter
- Создайте векторную базу и сохраниете ее на ГуглДиск
- Загрузите из ГуглДиска сохраненную на предыдущем шаге векторную базу
- Создайте простого консультанта по базе знаний. Используйте стандартную библиотеку openai.
- В этом задании не используйте ретриверы и ChatOpenAI из Langchain.
- Задайте вопрос консультанту по базе знаний и получите ответ

In [1]:
# Устанавливаем необходимые библиотеки
!pip install -q langchain==0.3.9 langchain_core==0.3.21 langchain_community==0.3.9 \
faiss-cpu==1.9.0.post1 tiktoken==0.8.0 sentence_transformers mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5

In [2]:
# Импортируем необходимые библиотеки
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer
import os
import re
import requests
import textwrap
import tiktoken
import matplotlib.pyplot as plt
from google.colab import userdata, drive
from mistralai import Mistral
from google.colab import userdata


api_key = userdata.get("MISTRAL_API_KEY")
os.environ["MISTRAL_API_KEY"] = api_key

# Создание клиента Mistral с использованием API ключа из переменных среды
client = Mistral(api_key)

# Ссылка на текстовый документ базы знаний
url = 'https://docs.google.com/document/d/1q4l912Re8zuIfBax4FDS3ZppYmVPzER3Si2wrmznddc'

In [22]:
# Функции
import numpy as np
import faiss

# функция для загрузки документа Docx по ссылке из гугл драйв
def download_google_doc(url: str) -> str:
    # Извлекаем ID документа из URL
    match_ = re.search(r'/document/d/([a-zA-Z0-9-_]+)', url)
    if not match_:
        raise ValueError("Invalid Google Docs URL")
    doc_id = match_.group(1)
    download_url = f'https://docs.google.com/document/d/{doc_id}/export?format=txt'
    response = requests.get(download_url, stream=True)
    if response.status_code != 200:
        raise RuntimeError("Failed to download the document")
    return response.text


# Функция для форматирования текста по абзацам
def format_text(text, width=120):
    # Разделяем текст на абзацы
    paragraphs = text.split('\n')
    # Форматируем каждый абзац отдельно
    formatted_paragraphs = []
    for paragraph in paragraphs:
        # Используем textwrap.fill для форматирования абзаца, чтобы длина строки не превышала width
        formatted_paragraph = textwrap.fill(paragraph, width)
        formatted_paragraphs.append(formatted_paragraph)
    # Объединяем абзацы с символом новой строки
    return '\n'.join(formatted_paragraphs)


# Функция возвращает количество токенов в строке в зависимости от используемой модели
def num_tokens_from_string(string: str, model='gpt-4o-mini') -> int:
    # Получаем имя кодировки для указанной модели
    encoding_name = tiktoken.encoding_for_model(model).name
    # Получаем объект кодировки на основе имени кодировки
    encoding = tiktoken.get_encoding(encoding_name)
    # Кодируем строку и вычисляем количество токенов
    num_tokens = len(encoding.encode(string))
    # Возвращаем количество токенов
    return num_tokens


# Построение гистограммы распределения количества токенов по чанкам
def create_histogram(chunks):
    print("\nОбщее количество чанков: ", len(chunks))
    # Подсчет токенов для каждого чанка
    try: # для формата чанков LangChain Document
        chunk_token_counts = [num_tokens_from_string(chunk.page_content) for chunk in chunks]
    except: # для текстового формата чанков
        chunk_token_counts = [num_tokens_from_string(chunk) for chunk in chunks]
    # Строим гистограмму
    plt.figure(figsize=(7, 4)) # размер
    plt.hist(chunk_token_counts, bins=10, alpha=0.5, label='Чанки')
    plt.title('Распределение к-ва токенов по чанкам')  # Заголовок графика
    plt.xlabel('К-во токенов в чанке')  # Подпись оси X
    plt.ylabel('К-во чанков')  # Подпись оси Y
    plt.show()  # Отображаем график


# (CharacterTextSplitter) Формируем чанки из текста по количеству символов
def split_text(text: str,
               chunk_size=3000,    # Ограничение к-ва символов в чанке
               chunk_overlap=200): # к-во символов перекрытия в чанке
    # Удалить пустые строки и лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    # Создаем экземпляр CharacterTextSplitter с заданными парамаетрами
    splitter = CharacterTextSplitter(chunk_size=chunk_size,
                                     chunk_overlap=chunk_overlap,
                                     separator=" ")
    # Разбиваем текст на чанки с помощью созданного сплиттера
    chunks = splitter.split_text(text)
    # Преобразование фрагментов текста в формат LangChain Document (без метаданных)
    chunks = [Document(page_content=chunk) for chunk in chunks]
    return chunks # Возвращаем список чанков в формате LangChain Document

# Функция для создания эмбеддингов из массива документов
def create_embeddings(documents):
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    page_contents = [doc.page_content for doc in documents]
    return model.encode(page_contents)

# Создание индексной (векторной) базы из текстовых чанков и сохранение на диск
def create_db_index_from_docs_save(chunks_docs,
                                   path):  # путь к папке
    # Создаем индексную базу с использованием FAISS ---------------------------------

    # Создание эмбеддингов
    embeddings = create_embeddings(chunks_docs)

    # Преобразование эмбеддингов в массив numpy
    embeddings = np.array(embeddings).astype('float32')

    # Создание индекса FAISS
    dimension = embeddings.shape[1]  # Размерность векторов
    db_index = faiss.IndexFlatL2(dimension)  # Использование L2 метрики
    # Добавление векторов в индекс
    db_index.add(embeddings)

    # сохраняем индексную базу ------------------------------------------------------
    faiss.write_index(db_index, path)
    # --------------------------------------------------------------------------------
    return db_index


# Загрузка векторной базы с диска
def load_db_vector(folder_path_db_index):           # имя сохраненной векторной базы
    return faiss.read_index(folder_path_db_index)


# Функция запроса и ответа от OpenAI с поиском по векторной базе данных
def generate_answer(query,    # запрос пользователя
                    db_index, # векторная база знаний
                    query_vector,
                    fragments,
                    k=5,      # используемое к-во чанков
                    verbose=True, # выводить ли на экран выбранные чанки
                    model='mistral-small-latest',
                    temp=0.1):

    # Поиск релевантных отрезков из базы знаний
    distances, indices =  db_index.search(query_vector, k=4)
    similar_documents = [fragments[i] for i in indices[0]]

    # Формирование текстового контента из выбранных чанков для модели
    message_content = re.sub(r'\n{2}', ' ', '\n '.join(
        [f'Отрывок документа № {i+1}:\n' + doc.page_content
        for i, doc in enumerate(similar_documents)]))
    if verbose:
        print(message_content) # печать на экран выбранных чанков

    messages = [{"role": "system",
                 "content": f'Ответь на вопрос пользователя на основании информации из базы знаний: \n{message_content}'},
                {"role": "user",
                 "content": f'Вопрос пользователя: {query}'}]
    client = Mistral(api_key)
    completion = client.chat.complete(
        model=model,
        messages=messages,
        temperature=temp
    )
    return completion.choices[0].message.content

# Решение

In [4]:
# База знаний, которая будет подаваться в langChain
source_chunks = split_text(format_text(download_google_doc(url)))
source_chunks

[Document(metadata={}, page_content='\ufeff"Правила безопасности опасных производственных объектов, на которых используются подъемные сооружения" I. Общие положения 1. Настоящие Федеральные нормы и правила в области промышленной безопасности "Правила безопасности опасных производственных объектов, на которых используются подъемные сооружения" (далее - ФНП) разработаны в соответствии с Федеральным законом от 21 июля 1997 г. N 116-ФЗ "О промышленной безопасности опасных производственных объектов" (Собрание законодательства Российской Федерации, 1997, N 30, ст.3588; 2018, N 31, ст.4860) (далее - Федеральный закон N 116-ФЗ). ФНП устанавливают необходимые требования к деятельности в области промышленной безопасности на опасных производственных объектах (далее - ОПО), на которых используются стационарно установленные грузоподъемные механизмы, подъемные сооружения (далее - ПС), в том числе к работникам указанных ОПО; Об утверждении федеральных норм и правил в области промышленной безопасности

In [15]:
create_db_index_from_docs_save(source_chunks, '/safety_of_lifting_equipment.idx')

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7a0c4bff6670> >

In [19]:
import pickle
# Сохранение документов на диск
with open('/safety_of_lifting_equipment_docs.pkl', 'wb') as f:
    pickle.dump(source_chunks, f)

In [20]:
# Загрузка документов с диска
with open('/safety_of_lifting_equipment_docs.pkl', 'rb') as f:
    documents = pickle.load(f)

# Проверка загруженных документов
for doc in documents:
    print(doc.page_content, doc.metadata)

﻿"Правила безопасности опасных производственных объектов, на которых используются подъемные сооружения" I. Общие положения 1. Настоящие Федеральные нормы и правила в области промышленной безопасности "Правила безопасности опасных производственных объектов, на которых используются подъемные сооружения" (далее - ФНП) разработаны в соответствии с Федеральным законом от 21 июля 1997 г. N 116-ФЗ "О промышленной безопасности опасных производственных объектов" (Собрание законодательства Российской Федерации, 1997, N 30, ст.3588; 2018, N 31, ст.4860) (далее - Федеральный закон N 116-ФЗ). ФНП устанавливают необходимые требования к деятельности в области промышленной безопасности на опасных производственных объектах (далее - ОПО), на которых используются стационарно установленные грузоподъемные механизмы, подъемные сооружения (далее - ПС), в том числе к работникам указанных ОПО; Об утверждении федеральных норм и правил в области промышленной безопасности "Правила безопасности опасных производств

In [16]:
db = load_db_vector('/safety_of_lifting_equipment.idx')

In [24]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
topic= "Какие требования предъявляются к промышленной безопасности на объектах, где используются подъемные сооружения?"                                      # Напишите сюда вопрос нейро-консультанту
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

ans = generate_answer(topic,db, query_vector, documents)
ans

Отрывок документа № 1:
в первоначальное положение, когда испытательный груз будет опущен. 173. Статические испытания кранов мостового типа, предназначенных для обслуживания электростанций, где невозможно (нецелесообразно) проведение испытаний с использованием грузов, могут проводиться при помощи специальных приспособлений, позволяющих создать испытательную нагрузку без применения груза. Порядок проведения таких испытаний должен быть определен инструкцией. При этом динамические испытания не проводятся. 174. Статические испытания крана стрелового типа, имеющего одну или несколько грузовых характеристик, при периодическом или внеочередном техническом освидетельствовании проводятся в положении, соответствующем наибольшей грузоподъемности крана и (или) наибольшему грузовому моменту, если это не противоречит требованиям, изложенным в руководстве (инструкции) по эксплуатации ПС. Испытания кранов, имеющих сменное стреловое оборудование, проводятся с установленным стреловым оборудованием. После

'Требования к промышленной безопасности на объектах, где используются подъемные сооружения, включают в себя множество аспектов, направленных на обеспечение безопасной эксплуатации и предотвращение аварийных ситуаций. Вот основные требования, которые можно выделить на основе предоставленных документов:\n\n1. **Испытания подъемных сооружений**:\n   - **Статические испытания**: Проводятся для проверки устойчивости и прочности конструкций. Например, для кранов стрелового типа испытания проводятся в положении, соответствующем наибольшей грузоподъемности и наибольшему грузовому моменту. Для кранов мостового типа испытания могут проводиться с использованием специальных приспособлений, создающих испытательную нагрузку без применения груза (документ № 1, пункты 173-175).\n   - **Динамические испытания**: Не проводятся для кранов мостового типа, если невозможно или нецелесообразно использовать грузы (документ № 1, пункт 173).\n\n2. **Контроль качества металлопроката и сварных соединений**:\n   -

In [25]:
topic= "Какие действия необходимо предпринять в случае аварии или инцидента на опасном производственном объекте?"                                      # Напишите сюда вопрос нейро-консультанту
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

ans = generate_answer(topic,db, query_vector, documents)
ans

Отрывок документа № 1:
технического освидетельствования смонтированного ПС, выполненного в соответствии с пунктами 164-190 настоящих ФНП. 61. Не допускается выполнять монтаж и наладку ПС с нарушениями требований руководства (инструкции) по эксплуатации ПС, а также требований настоящих ФНП. 62. Ответственной за приведение в соответствие ПС, изготовленного по ранее разработанным проектам и не Об утверждении федеральных норм и правил в области промышленной безопасности "Правила безопасности опасных производственных объектов, на которых используются подъемные сооружения" Приказ Ростехнадзора от 26.11.2020 N 461 ФНП в области промышленной безопасности от 26.11.2020 N 461 Страница 14 Интранет оборудованного ограничителями, указателями и регистраторами, необходимыми для обеспечения промышленной безопасности технологического процесса, в котором используется ПС, либо продолжение эксплуатации ПС со снижением его паспортных показателей назначения является эксплуатирующая организация. V. Ремонт, р

'В случае аварии или инцидента на опасном производственном объекте (ОПО) необходимо предпринять следующие действия:\n\n1. **Обеспечение безопасности**: Незамедлительно прекратить выполнение работ и эвакуировать персонал из опасной зоны. Обеспечить безопасность всех работников и предотвратить распространение аварии.\n\n2. **Уведомление ответственных лиц**: Сообщить о происшествии руководству организации, ответственным за безопасность на ОПО, а также в соответствующие государственные органы, такие как Ростехнадзор.\n\n3. **Оценка ущерба**: Провести оценку ущерба и повреждений, вызванных аварией или инцидентом. Это может включать оценку состояния оборудования, инфраструктуры и окружающей среды.\n\n4. **Документирование**: Зафиксировать все обстоятельства аварии или инцидента, включая время, место, причины, последствия и принятые меры. Вести подробный журнал событий и действий, предпринятых для устранения последствий.\n\n5. **Устранение последствий**: Принять меры для устранения последстви

In [27]:
topic= "Какие принципы промышленной безопасности должны соблюдаться для предотвращения аварий и инцидентов?"                                      # Напишите сюда вопрос нейро-консультанту
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

ans = generate_answer(topic,db, query_vector, documents)
ans

Отрывок документа № 1:
выполняться в соответствии с проектом ремонта, реконструкции или модернизации ПС, разработанным специализированной организацией. При этом суммарная длина контролируемых участков сварных соединений устанавливается специализированной организацией в проекте ремонта, реконструкции или модернизации ПС и должна составлять не менее: 50 процентов от длины стыка - на каждом стыке растянутого пояса коробчатой или ферменной металлоконструкции; 25 процентов от длины стыка - для всех остальных стыковых соединений. Ремонтные сварные соединения элементов металлоконструкций из высокопрочных сталей подвергаются 100-процентному ультразвуковому и магнитопорошковому контролю. Допускается замена магнитопорошкового контроля на капиллярный контроль. Применение капиллярного контроля сварных швов (кроме стыковых) устанавливается специализированной организацией в проекте ремонта, реконструкции или модернизации ПС. 79. Качество ремонтных сварных соединений считается неудовлетворительным, е

'Для предотвращения аварий и инцидентов при эксплуатации подъемных сооружений (ПС) должны соблюдаться следующие принципы промышленной безопасности:\n\n1. **Соблюдение эксплуатационной документации**: Все работы по эксплуатации, ремонту и техническому обслуживанию ПС должны выполняться в соответствии с руководством (инструкцией) по эксплуатации ПС. Это включает в себя выполнение плановых и внеочередных ремонтов, а также проведение испытаний и проверок (Отрывок документа № 1, пункт 81; Отрывок документа № 2, пункт 173).\n\n2. **Контроль качества сварных соединений**: Ремонтные сварные соединения должны подвергаться неразрушающему контролю, включая ультразвуковой и магнитопорошковый контроль. При выявлении дефектов, выходящих за пределы норм, они должны быть исправлены с последующим подтверждением качества соединения (Отрывок документа № 1, пункты 79-80).\n\n3. **Проверка и испытания ПС**: Статические и динамические испытания ПС должны проводиться в соответствии с требованиями руководства